# Backtesting ML Classification-Based

In [ ]:
???

![](src/res_classification.png)

## Load the model

In [16]:
import pickle

In [17]:
with open('models/model_dt_classifier.pkl', 'rb') as f:
    my_model_dt = pickle.load(f)

with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [18]:
my_model_dt

DecisionTreeClassifier(max_depth=12)

## Load the data

In [32]:
import pandas as pd

df = pd.read_excel('data/microsoft-linkedin-processed.xlsx', index_col=0)
# df.index = [d.split(" ")[0] for d in df.index]
df.index = pd.to_datetime(df.index)
df

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction
Price,,,,,,,
2016-12-01,59.200001,60.150002,58.939999,60.110001,34542100,0.084387,UP
2016-12-02,59.250000,59.470001,58.799999,59.080002,25515700,1.610763,UP
2016-12-05,60.220001,60.590000,59.560001,59.700001,23552700,-0.450376,DOWN
2016-12-06,59.950001,60.459999,59.799999,60.430000,19907000,2.313831,UP
2016-12-07,61.369999,61.380001,59.799999,60.009998,30809000,-0.590068,DOWN
...,...,...,...,...,...,...,...
2024-11-04,408.459991,410.420013,405.570007,409.799988,19672300,0.729111,UP
2024-11-05,411.459991,414.899994,408.079987,408.369995,17626000,2.075301,UP
2024-11-06,420.179993,420.450012,410.519989,412.420013,26681800,1.234046,UP


## Backtesting.py Library

### Create your Strategy Class

In [20]:
from backtesting import Backtest, Strategy

In [33]:
df_explanatory = df.drop(columns=["change_tomorrow","change_tomorrow_direction"])
df_explanatory
df_target = df.change_tomorrow_direction
df_target

Price
2016-12-01      UP
2016-12-02      UP
2016-12-05    DOWN
2016-12-06      UP
2016-12-07    DOWN
              ... 
2024-11-04      UP
2024-11-05      UP
2024-11-06      UP
2024-11-07    DOWN
2024-11-08    DOWN
Name: change_tomorrow_direction, Length: 1998, dtype: object

In [36]:
(my_model_dt.predict(X=df_explanatory) == df_target).sum()/len(df_target)

0.7182182182182182

In [23]:
explanatory_today = df_explanatory.iloc[[-1],:]

In [24]:
forecast_tomorrow =  my_model_dt.predict(explanatory_today)[0]

In [37]:
list_buy_sell = [1 if d=='UP' else -1 for d in df['change_tomorrow_direction']]
df['buy_sell'] = list_buy_sell
df.head(15)

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction,buy_sell
Price,,,,,,,,
2016-12-01,59.200001,60.150002,58.939999,60.110001,34542100,0.084387,UP,1
2016-12-02,59.250000,59.470001,58.799999,59.080002,25515700,1.610763,UP,1
2016-12-05,60.220001,60.590000,59.560001,59.700001,23552700,-0.450376,DOWN,-1
2016-12-06,59.950001,60.459999,59.799999,60.430000,19907000,2.313831,UP,1
2016-12-07,61.369999,61.380001,59.799999,60.009998,30809000,-0.590068,DOWN,-1
2016-12-08,61.009998,61.580002,60.840000,61.299999,21220800,1.549141,UP,1
2016-12-09,61.970001,61.990002,61.130001,61.180000,27349400,0.321694,UP,1
2016-12-12,62.169998,62.299999,61.720001,61.820000,20198100,1.286125,UP,1
2016-12-13,62.980000,63.419998,62.240002,62.500000,35718900,-0.478620,DOWN,-1


#### Simulate the prediction for the last observation

#### Write the prediction process in the Strategy class

#### Calculate Purchase Recommendation

##### Buy if it goes up

##### Buy if it goes and sell if down

> You can only sell if you have already bought

In [39]:
list_buy_sell = []
already_bought = False
series_up = 0
series_down = 0

for d in df.change_tomorrow_direction:
  action = 0
  if d=='UP':
    series_up+=1
    series_down=0
    if series_up>2 and already_bought == False:
      action = 1
      already_bought = True
  elif d=='DOWN':
    series_up = 0
    series_down+=1
    if series_down>2 and already_bought == True:
      action = -1
      already_bought = False
  list_buy_sell.append(action)

df['buy_sell_track'] = list_buy_sell
df[['change_tomorrow_direction', 'buy_sell','buy_sell_track']].head(15).tail(10)

,change_tomorrow_direction,buy_sell,buy_sell_track
Price,,,
2016-12-08,UP,1,0
2016-12-09,UP,1,0
2016-12-12,UP,1,1
2016-12-13,DOWN,-1,0
2016-12-14,DOWN,-1,0
2016-12-15,DOWN,-1,-1
2016-12-16,UP,1,0
2016-12-19,DOWN,-1,0
2016-12-20,DOWN,-1,0


#### Add conditions to the strategy

In [60]:
class ClassificationUP(Strategy):
    def init(self):
        self.model = my_model_dt
        self.already_bought = False
        self.series_up = 0
        self.series_down = 0
        self.up_streak = 0
        self.down_streak = 0

    def next(self):
        explanatory_today = self.data.df.iloc[[-1],:]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        # conditions to sell or buy
        if forecast_tomorrow == 'UP':
            self.series_up +=1
            self.series_down = 0
            if self.series_up>self.up_streak and self.already_bought == False:
                self.buy()
                self.already_bought = True
        elif forecast_tomorrow == 'DOWN':
            self.series_up = 0
            self.series_down += 1
            if self.series_down > self.down_streak and self.already_bought == True:
                self.sell()
                self.already_bought = False 


        

### Define initial conditions

In [61]:
bt = Backtest(data=df_explanatory, strategy=ClassificationUP,
              cash=100, commission=.002, exclusive_orders=True)

/tmp/ipykernel_25401/2254245403.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data=df_explanatory, strategy=ClassificationUP,


### Run backtesting

In [63]:
results = bt.run()
results

Start                     2016-12-01 00:00:00
End                       2024-11-08 00:00:00
Duration                   2899 days 00:00:00
Exposure Time [%]                     99.8999
Equity Final [$]                  29623.62607
Equity Peak [$]                   29689.90922
Return [%]                        29523.62607
Buy & Hold Return [%]              613.750005
Return (Ann.) [%]                  104.992586
Volatility (Ann.) [%]               48.850046
Sharpe Ratio                         2.149283
Sortino Ratio                        7.368728
Calmar Ratio                         3.599519
Max. Drawdown [%]                  -29.168505
Avg. Drawdown [%]                   -1.838283
Max. Drawdown Duration      286 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                  578
Win Rate [%]                         67.99308
Best Trade [%]                      13.806898
Worst Trade [%]                    -12.433073
Avg. Trade [%]                    

### Interpret backtesting results

In [64]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-01 00:00:00
End,2024-11-08 00:00:00
Duration,2899 days 00:00:00
Exposure Time [%],99.8999
Equity Final [$],29623.62607
Equity Peak [$],29689.90922
Return [%],29523.62607


## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="03C_Backtesting ML Classification-Based.ipynb">notebook</a>.